In [ ]:
!docker stop minimal-django-postgres
!docker rm minimal-django-postgres

In [ ]:
!docker run -e POSTGRES_PASSWORD=postgres -d -p 5432:5432 --name minimal-django-postgres postgres

In [ ]:
!mkdir -p /tmp/dj_full_cycle/tmp
%cd /tmp/dj_full_cycle

In [ ]:
import sys
import django
from django.conf import settings
from django.db import connection, models

urlpatterns = []

settings.configure(
#     DATABASES={'default': {'ENGINE': 'django.db.backends.sqlite3', 'NAME': ':memory:'}},
    DATABASES={'default': {'ENGINE': 'django.db.backends.postgresql', 'USER': 'postgres', 'NAME': 'postgres', 'PASSWORD': 'postgres', 'HOST': '127.0.0.1', 'PORT': '5432'}},
    INSTALLED_APPS=['django.contrib.contenttypes', 'django.contrib.auth', 'rest_framework', 'django_filters', 'tmp'],
    ROOT_URLCONF=sys.modules[__name__],
    REST_FRAMEWORK={'DEFAULT_FILTER_BACKENDS': ['django_filters.rest_framework.DjangoFilterBackend']},
    DEBUG=True,
    ALLOWED_HOSTS=['*'],
)
django.setup()

In [ ]:
# # to change model, (and then create migrations again)
# from django.apps import apps
# del apps.all_models['tmp']['mymodel']

class MyModel(models.Model):
    name = models.CharField(max_length=256)

    class Meta:
        app_label = 'tmp'

In [ ]:
# !rm -rf ./tmp/migrations/
# # to create models through migrations:
# from django.core import management
# management.call_command('makemigrations', 'tmp')
# management.call_command('migrate', 'tmp')
# # to create tables using schema_editor
with connection.schema_editor() as schema_editor:
    schema_editor.create_model(MyModel)

# defining views

In [ ]:
import re
from rest_framework import viewsets
from rest_framework import serializers
import django_filters

In [ ]:
class MyModelFilter(django_filters.FilterSet):
    class Meta:
        model = MyModel
        fields = '__all__'
        
class MyModelSerializer(serializers.ModelSerializer):
    class Meta:
        model = MyModel
        fields = '__all__'

class MyModelViewSet(viewsets.ModelViewSet):
    queryset = MyModel.objects.all()
    serializer_class = MyModelSerializer
    filter_class = MyModelFilter

# urls

In [ ]:
from rest_framework.routers import DefaultRouter
from django.urls import path, include
from django.test import Client
from IPython.display import HTML

c = Client()

In [ ]:
router = DefaultRouter()
router.register('mymodel', MyModelViewSet)
mypath = path('', include(router.urls))
if urlpatterns:
    urlpatterns[0] = mypath
else:
    urlpatterns.append(mypath)

In [ ]:
HTML(c.get('/mymodel/?name=xxx').content.decode('utf8'))

In [ ]:
MyModel.objects.create(name='test')